In [1]:
%run installations.ipynb

     |████████████████████████████████| 49.5 MB 9.9 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 4.6 MB 18.6 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 9.7 MB 11.3 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Start writing code here...

import open3d as o3d
import copy
import numpy as np
import time
import glob





def load_point_clouds(pcl_path="./rotary_room_scanner/rotary_rsrc/clouds3/", voxel_size=0.02, step_size=10):
    pcls = []

    list_of_pcls = glob.glob(pcl_path + '*.ply')
    list_of_pcls.sort()
    for pcl_filepath in list_of_pcls[::step_size]:
        pcl = o3d.io.read_point_cloud(pcl_filepath)
        pcl_down = pcl.voxel_down_sample(voxel_size=voxel_size)
        pcls.append(pcl_down)
    return pcls

# The following two functions are taken from http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html
def preprocess_point_cloud(voxel_size, pc1):
    radius_normal = voxel_size * 10
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pc1, _ = pc1.remove_statistical_outlier(nb_neighbors=40, std_ratio=0.5)
    pc1.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    # radius_feature = voxel_size * 5
    # print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    # pc1_fpfh = o3d.registration.compute_fpfh_feature(
    #     pc1,
    #     o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    # return pc1 with estimated normal info & fpfh feature vectors (33 feat)
    return pc1


def pairwise_registration(source, target, voxel_size=0.02):
    print("Apply point-to-plane ICP")
    max_correspondence_distance_coarse = voxel_size * 15
    max_correspondence_distance_fine = voxel_size * 1.5

    source = preprocess_point_cloud(voxel_size, source)
    target = preprocess_point_cloud(voxel_size, target)
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp



def multiway_registration(pcls):
    # Reference: http://www.open3d.org/docs/latest/tutorial/Advanced/multiway_registration.html
    #input: all the pointclouds
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcls)


    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcls[source_id], pcls[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph



cl_path="rotary/clouds3/"
voxel_size=0.02
step_size=4


pcds_down = load_point_clouds(pcl_path="./rotary_room_scanner/rotary_rsrc/clouds3/", voxel_size=0.02,step_size=step_size)
print(f"Trying to merge  {len(pcds_down)} point clouds")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = multiway_registration(pcds_down)


print("Optimizing PoseGraph ...")
# Set options for global optimization of pose graph
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=voxel_size*1.5,
    edge_prune_threshold=0.25,
    reference_node=0)

# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     o3d.registration.global_optimization(
#         pose_graph,
#         o3d.registration.GlobalOptimizationLevenbergMarquardt(),
#         o3d.registration.GlobalOptimizationConvergenceCriteria(),
#         option)
print("Entering global optim")
o3d.registration.global_optimization(
        pose_graph,
        o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(),
        option)


print("Entering point cloud merging")

pcd_combined = o3d.geometry.PointCloud()

for point_id in range(len(pcds_down)):
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds_down[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud(f"multiway_registration{time.time()}.ply", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down])



Apply point-to-plane ICP
:: Estimate normal with search radius 0.200.
[Open3D DEBUG] Pointcloud down sampled from 8267 points to 8018 points.
:: Estimate normal with search radius 0.200.
[Open3D DEBUG] Pointcloud down sampled from 12849 points to 12771 points.
[Open3D DEBUG] ICP Iteration #0: Fitness 0.5678, RMSE 0.1498
[Open3D DEBUG] Residual : 2.16e-02 (# of elements : 4553)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.6663, RMSE 0.1013
[Open3D DEBUG] Residual : 9.67e-03 (# of elements : 5342)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.6987, RMSE 0.0773
[Open3D DEBUG] Residual : 5.14e-03 (# of elements : 5602)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.7277, RMSE 0.0944
[Open3D DEBUG] Residual : 7.35e-03 (# of elements : 5835)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.8285, RMSE 0.1264
[Open3D DEBUG] Residual : 1.42e-02 (# of elements : 6643)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.9154, RMSE 0.1333
[Open3D DEBUG] Residual : 1.52e-02 (# of elements : 7340)
[Open3D DEBUG] ICP Iterat

[Open3D DEBUG] Residual : 2.09e-04 (# of elements : 2041)
[Open3D DEBUG] ICP Iteration #29: Fitness 0.2754, RMSE 0.0177
[Open3D DEBUG] Residual : 2.12e-04 (# of elements : 2058)
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
:: Estimate normal with search radius 0.200.
[Open3D DEBUG] Pointcloud down sampled from 7990 points to 7472 points.
:: Estimate normal with search radius 0.200.
[Open3D DEBUG] Pointcloud down sampled from 10445 points to 10386 points.
[Open3D DEBUG] ICP Iteration #0: Fitness 0.6701, RMSE 0.1483
[Open3D DEBUG] Residual : 1.85e-02 (# of elements : 5007)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.7492, RMSE 0.0874
[Open3D DEBUG] Residual : 4.51e-03 (# of elements : 5598)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.7793, RMSE 0.0907
[Open3D DEBUG] Residual : 4.60e-03 (# of elements : 5823)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.8009, RMSE 0.0939
[Open3D DEBUG] Residual : 4.95e-03 (# of elements : 5984)
[Open3D DEBUG] ICP Iteration #4: Fitness 0

[Open3D DEBUG] Residual : 5.17e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #14: Fitness 1.0000, RMSE 0.0432
[Open3D DEBUG] Residual : 5.08e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #15: Fitness 1.0000, RMSE 0.0447
[Open3D DEBUG] Residual : 5.03e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #16: Fitness 1.0000, RMSE 0.0456
[Open3D DEBUG] Residual : 5.01e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #17: Fitness 1.0000, RMSE 0.0462
[Open3D DEBUG] Residual : 5.00e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #18: Fitness 1.0000, RMSE 0.0466
[Open3D DEBUG] Residual : 4.99e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #19: Fitness 1.0000, RMSE 0.0468
[Open3D DEBUG] Residual : 5.00e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #20: Fitness 1.0000, RMSE 0.0469
[Open3D DEBUG] Residual : 5.00e-04 (# of elements : 16545)
[Open3D DEBUG] ICP Iteration #21: Fitness 1.0000, RMSE 0.0469
[Open3D DEBUG] Residual : 5.00e-

RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m